# Interest Rate Risk I: Duration

## Bond price plot

In [1]:
import numpy as np
import pandas as pd
import numpy_financial as npf
import plotly.graph_objects as go

In [2]:
def bondpv(cr,ytm,nop,ttm, principal):
    ''' 
    THIS IS A COMMENT SECTION
    WHEN WRITING FUNCTIONS, BEST PRACTICE IS TO DESCRIBE THE INPUTS AND OUTPUTS
    Inputs:
        cr:             coupon rate (per year, in decimal)
        ytm:            yield-to-maturity (per year, in decimal)
        nop:            number of payments per year
        ttm:            time-to-maturity (in periods)
        principal:      face value
    Outputs:
        p:              bond price
    '''
    dr = ytm/nop
    c = cr/nop*principal
    p = -npf.pv(dr,ttm,c,principal)
    return p

In [3]:
# Calculate bond prices for a range of yields
# Inputs
CR = 0.06           # annual coupon rate
NOP= 2              # payments per year
PRINCIPAL = 100     # face value of the bond
TTM_LIST  = [8, 16, 40]
YIELDS    = np.arange(0.005,0.125,0.005)

# Create dataframe of bond prices for each ttm and yield
df = pd.DataFrame(dtype=float,columns = TTM_LIST,index=YIELDS)
df.index.name='ytm'
df.columns
for t in TTM_LIST:
    for y in YIELDS:
        df.loc[y,t] = bondpv(CR,y,NOP,t,PRINCIPAL)
df

,8,16,40
ytm,,,
0.005,121.754548,143.078860,204.554624
0.010,119.557398,138.349813,190.430570
0.015,117.407380,133.804703,177.505611
0.020,115.303356,129.435748,165.669372
0.025,113.244218,125.235511,154.822133
0.030,111.228888,121.196896,144.873768
0.035,109.256316,117.313121,135.742787
0.040,107.325481,113.577709,127.355479
0.045,105.435388,109.984473,119.645142


In [4]:
# Plot the data
fig = go.Figure()
for t in TTM_LIST:
    ttm_years = int(t/NOP)
    trace= go.Scatter(x=df.index, y=df[t], mode='lines', name=str(ttm_years)+' years to maturity')
    fig.add_trace(trace)

fig.update_xaxes(title='Yield',tickformat=".2%")
fig.update_yaxes(title='Price',tickformat=".2f")
fig.update_layout(title='Bond Pricing')
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.8))
fig.show()

## Duration

### Method #1: Brute force method

In [5]:
# Create dataframe to hold cash flows
TTM_PERIODS = TTM_LIST[0]
YTM = 0.05
bond = pd.DataFrame(dtype=float,columns=['Time (years)', 'Cash Flow', 'PV(Cash Flow)','Fraction of PV', 'Time*Fraction'], index=1+np.arange(TTM_PERIODS))
bond['Time (years)'] = bond.index / NOP
bond.index.name = 'Period'
bond['Cash Flow'] = CR / NOP * PRINCIPAL
bond.loc[TTM_PERIODS,'Cash Flow'] = bond.loc[TTM_PERIODS,'Cash Flow'] + PRINCIPAL
bond['PV(Cash Flow)'] = bond['Cash Flow'] / (1+YTM/NOP)**bond.index
bond

,Time (years),Cash Flow,PV(Cash Flow),Fraction of PV,Time*Fraction
Period,,,,,
1,0.5,3.0,2.926829,NaN,NaN
2,1.0,3.0,2.855443,NaN,NaN
3,1.5,3.0,2.785798,NaN,NaN
4,2.0,3.0,2.717852,NaN,NaN
5,2.5,3.0,2.651563,NaN,NaN
6,3.0,3.0,2.586891,NaN,NaN
7,3.5,3.0,2.523796,NaN,NaN
8,4.0,103.0,84.536897,NaN,NaN


In [6]:
# Calculate each time period's contribution to overall price
price = bond['PV(Cash Flow)'].sum()
bond['Fraction of PV'] = bond['PV(Cash Flow)'] / price
bond['Time*Fraction'] = bond['Time (years)']*bond['Fraction of PV']
bond

,Time (years),Cash Flow,PV(Cash Flow),Fraction of PV,Time*Fraction
Period,,,,,
1,0.5,3.0,2.926829,0.028255,0.014128
2,1.0,3.0,2.855443,0.027566,0.027566
3,1.5,3.0,2.785798,0.026894,0.040341
4,2.0,3.0,2.717852,0.026238,0.052476
5,2.5,3.0,2.651563,0.025598,0.063995
6,3.0,3.0,2.586891,0.024974,0.074921
7,3.5,3.0,2.523796,0.024364,0.085276
8,4.0,103.0,84.536897,0.816111,3.264443


In [7]:
# Check weights sum to 1
bond['Fraction of PV'].sum()

1.0

In [8]:
# Calculate duration
duration = bond['Time*Fraction'].sum()
duration

3.6231448727704616

In [9]:
# Plot PV weights of each time period
trace = go.Bar(x=bond['Time (years)'], y=bond['Fraction of PV'])
fig = go.Figure()
fig.add_trace(trace)
fig.update_xaxes(title='Time of CF (in years)',tickformat=".1f")
fig.update_yaxes(title='Fraction of Price',tickformat=".2f")
fig.add_vline(x=duration, line_width=4, line_dash="dash", line_color="black")
fig.show()

### Method #2: Custom Function


In [10]:
def duration(cr,ytm,nop,ttm, principal):
    ''' 
    Inputs:
        cr:             coupon rate (per year, in decimal)
        ytm:            yield-to-maturity (per year, in decimal)
        nop:            number of payments per year
        ttm:            time-to-maturity (in periods)
        principal:      face value
    Outputs:
        duration:       Macauley duration
    '''
    dr = ytm/nop
    c = cr/nop*principal
    p = -npf.pv(dr,ttm,c,principal)
    # Write out CFs and discount
    cfs = c*np.ones(ttm)
    cfs[-1] = cfs[-1]+principal
    pvs = cfs/(1+dr)**(1+np.arange(ttm))
    # Calculate weights
    wgts = pvs / np.sum(pvs)
    t = (1+np.arange(ttm))/nop
    # Calculate duration
    duration = t @ wgts
    return duration
duration(CR,YTM,NOP,TTM_PERIODS,PRINCIPAL)

3.6231448727704616

In [11]:
def mduration(cr,ytm,nop,ttm, principal):
    ''' 
    Inputs:
        cr:             coupon rate (per year, in decimal)
        ytm:            yield-to-maturity (per year, in decimal)
        nop:            number of payments per year
        ttm:            time-to-maturity (in periods)
        principal:      face value
    Outputs:
        duration:       modified duration
    '''
    dr = ytm/nop
    c = cr/nop*principal
    p = -npf.pv(dr,ttm,c,principal)
    # Write out CFs and discount
    cfs = c*np.ones(ttm)
    cfs[-1] = cfs[-1]+principal
    pvs = cfs/(1+dr)**(1+np.arange(ttm))
    # Calculate weights
    wgts = pvs / np.sum(pvs)
    t = (1+np.arange(ttm))/nop
    # Calculate duration
    duration = t @ wgts
    return duration / (1+dr)
mduration(CR,YTM,NOP,TTM_PERIODS,PRINCIPAL)
mduration(0.06,0.05,1,5,100)

4.26452499341714

### Approximate price change

For a change in yield $y$, the percent change in price is:
$$\frac{\Delta P}{P} \approx -D \cdot \frac{\Delta y}{1+y}.$$
Or equivalently, in terms of modified duration:
$$\frac{\Delta P}{P} \approx -D_{\text{modified}} \cdot \Delta y.$$

In [12]:
def implied_pct_chg(cr,ytm,nop,ttm, principal, delta_ytm):
    dmod = mduration(cr,ytm,nop,ttm, principal)
    return -dmod * delta_ytm

implied_pct_chg(0.06,0.05,1,5,100,-0.015)


0.0639678749012571

In [13]:
def actual_pct_chg(cr,ytm,nop,ttm, principal, delta_ytm):
    p = bondpv(cr,ytm,nop,ttm, principal)
    pnew = bondpv(cr,ytm+delta_ytm,nop,ttm, principal)
    return pnew/p - 1
actual_pct_chg(0.06,0.05,1,5,100,-0.015)
   

0.06669403978717336

### Testing the approximation

In [14]:
CR  = 0.05          # annual coupon rate
YTM = 0.10          # annual yield to maturity
NOP = 1             # payments per year
PRINCIPAL = 100     # face value of the bond
TTM_LIST  = [10, 20]
DELTA_YIELDS    = np.arange(-0.02,0.025,0.005)
column_index = pd.MultiIndex.from_product([['actual_pct_chg','approx_pct_chg'],TTM_LIST])
df = pd.DataFrame(dtype=float,columns=column_index,index=DELTA_YIELDS)
df.index.name='delta_ytm'

In [15]:
df['new_ytm'] = YTM + df.index
for dy in df.index:
    for ttm in TTM_LIST:
        df.loc[dy,('actual_pct_chg',ttm)] = actual_pct_chg(CR,YTM,NOP,ttm,PRINCIPAL,dy)
        df.loc[dy,('approx_pct_chg',ttm)] = implied_pct_chg(CR,YTM,NOP,ttm,PRINCIPAL,dy)
pd.options.display.float_format = '{:,.4f}'.format        
df

actual_pct_chg         approx_pct_chg         new_ytm
                      10      20             10      20        
delta_ytm                                                      
-0.0200           0.1529  0.2283         0.1393  0.1953  0.0800
-0.0150           0.1120  0.1645         0.1045  0.1465  0.0850
-0.0100           0.0729  0.1054         0.0696  0.0976  0.0900
-0.0050           0.0356  0.0507         0.0348  0.0488  0.0950
0.0000            0.0000  0.0000        -0.0000 -0.0000  0.1000
0.0050           -0.0340 -0.0470        -0.0348 -0.0488  0.1050
0.0100           -0.0666 -0.0908        -0.0696 -0.0976  0.1100
0.0150           -0.0977 -0.1314        -0.1045 -0.1465  0.1150
0.0200           -0.1274 -0.1692        -0.1393 -0.1953  0.1200

In [16]:
# Plot the data
fig = go.Figure()
for ttm in TTM_LIST:
    ttm_years = int(ttm/NOP)
    trace_act=go.Scatter(x=df.index, y=df[('actual_pct_chg',ttm)], mode='lines', name=str(ttm_years)+' years to maturity: Actual % Change')
    trace_approx=go.Scatter(x=df.index, y=df[('approx_pct_chg',ttm)], mode='lines', name=str(ttm_years)+' years to maturity: Approx % Change')
    fig.add_trace(trace_act)
    fig.add_trace(trace_approx)

fig.update_xaxes(title='Change in Yield',tickformat=".2%")
fig.update_yaxes(title='% Change in Price',tickformat=".2%")
fig.update_layout(title='Interest Rate Risk')
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.6))
fig.show()